Welcome to the PySCF tutorial!

This component of the tutorial is primarily concerned with computational chemistry calculations and will not cover the theoretical details of the methods utilized [i.e., Hartree--Fock (HF), Kohn--Sham Density Functional Theory (KS-DFT), Møller–Plesset Peturbation Theory (MP), Coupled Cluster Theory (CC), etc.]. For specifics regarding these methods, please see [TODO].

We will go through the steps necessary to carry out a variety of relevant calculations. These include:

* Single Point Energies
  * Ground State
    1. atomization energies and bond dissociation energies
    2. reaction energies
    3. electron affinities, ionization potentials, and proton affinities (vertical and adiabatic)
    4. isomerization (relative) energies
    5. binding energies of non-covalent complexes (weak/hydrogen/rare-gas)
    6. barrier heights
  * Excited State (Time-Dependent)
    1. TD-HF/TD-DFT
    2. Wavefunction Stability Analysis
    3. Visualization?

* Forces and Geometry Optimizations
  * Ground State
      1. Force for equilibrium structure to show zeros
      2. Force for non-eq
      3. TS search
      4. Geom opt
  * Excited State (Time-Dependent)
      1. can this be done?

* Frequencies
  * Ground State
    1. vibrational modes
    2. free-energy
    3. Geometry optimization local minimum
  * Excited State (Time-Dependent)
    1. can this be done?

Furthermore, we will demonstrate the effect of the most important computational settings on the final calculated results using a handful of the above interactions:
  * Relevant Settings
    1. basis set (minimal vs. double vs. triple. vs. quadruple vs. basis set limit) BSIE
    2. DFT integration grid (coarse vs. fine, etc.)
    3. method (HF/DFT/MP2/CC/etc.)
    4. counterpoise corrections (which is an extension of basis set category) talk about BSIE/BSSE
    5. integral threshes and convergence criteria
    6. cart vs spherical?
    7. restricted vs unrestricted
    8. density fitting
    9. opt algo, like diis gdm Newton etc

---

In order to get started, we need to import the pyscf module, which is accomplished by the command in the Code cell below. If this command fails, please see the [quick setup guide](http://sunqm.github.io/pyscf/tutorial.html#quick-setup) or the [detailed installation instructions](http://sunqm.github.io/pyscf/install.html).

In [1]:
import pyscf

The next step is to define the molecule, which requires importing the gto (gaussian type orbital) submodule:

In [2]:
from pyscf import gto

# Creating a molecule (mol) object with gto.Mole()

We will now go through the steps involved in setting up a molecule, including a description of the eight most important attributes. The following Code cell combines the content covered here into a useable sample input. The more advanced attributes are listed (commented) at the bottom of the Code cell and the advanced user can find more information about them [here](http://sunqm.github.io/pyscf/gto.html).

The molecule used in this demonstration is the water dimer (two hydrogen-bonded water molecules) from the widely-used [S22 dataset](http://pubs.rsc.org/en/Content/ArticleLanding/2006/CP/B600027D#!divAbstract) of non-covalent interactions, and can be found [here](http://www.begdb.com/index.php?action=oneMolecule&state=show&id=82). The molecule is pictured here:

<img style="float: left;" src="images/water_dimer.jpg"><br clear="all" />

The first step is to create the molecule object, mol:
```
mol=gto.Mole()
```
Note: The molecule object can be named nearly anything (e.g., mol1, mol2, mymol, etc.).

## atom attribute
#### (NO DEFAULT)

PySCF allows for a variety of [molecular input formats](http://sunqm.github.io/pyscf/gto.html#geometry), but the one that is most suitable for the current example is the use of triple-quotes. This allows one to simply copy and paste the geometry from the BEGDB website without further modification:
```
mol.atom="""
O  -1.551007  -0.114520   0.000000
H  -1.934259   0.762503   0.000000
H  -0.599677   0.040712   0.000000
O   1.350625   0.111469   0.000000
H   1.680398  -0.373741  -0.758561
H   1.680398  -0.373741   0.758561"""
```
Another way to set up a molecule is to read the coordinates from a file on disk. The file can be formatted in three unique ways:

1. XYZ (see geom/water_dimer.xyz)
2. Z-matrix (see geom/water_dimer.zmat) [TODO]
3. Cartesian coordinates only (see geom/water_dimer.mol)
4. Cartesian coordinates prepended by a line containing the charge and spin separated by a space (see geom/water_dimer.qc)

As long as the file adheres to one of these formats, PySCF can automatically detect the format and parse the file regardless of the extension. Thus, reading in a molecule file can be accomplished simply via [TODO]:
```
mol.atom=read_file('water_dimer.mol')
```

## basis attribute
#### (NO DEFAULT)

The simplest way to set up the basis is to specify the same basis set for all atoms:
```
mol.basis='aug-cc-pVDZ'
```
In this case, the aug-cc-pVDZ basis set is used for all of the hydrogen and oxygen atoms.

Alternatively, one can use a different basis set for different elements by using Python's dictionary (dict) data structure:
```
mol.basis={'H': 'cc-pVDZ', 'O': 'aug-cc-pVDZ'}
```
In this case, the cc-pVDZ basis set is used for all of the hydrogen atoms, while the diffuse aug-cc-pVDZ basis set is used for all of the oxygen atoms.

One can also use different basis sets for different atoms of the same element by labeling the atoms with integers:
```
mol.atom="""
O1  -1.551007  -0.114520   0.000000
H2  -1.934259   0.762503   0.000000
H3  -0.599677   0.040712   0.000000
O4   1.350625   0.111469   0.000000
H5   1.680398  -0.373741  -0.758561
H6   1.680398  -0.373741   0.758561"""
mol.basis={'O1': 'cc-pVDZ', 'H2': 'cc-pVDZ', 'H3': 'aug-cc-pVDZ', 'O4': 'aug-cc-pVDZ', 'H5': 'cc-pVDZ', 'H6': 'cc-pVDZ'}
```
In this case, the cc-pVDZ basis set is used for all of the atoms except for those involved in the hydrogen bond (H3 and O4). The diffuse aug-cc-pVDZ basis set is used for H3 and O4.

Finally, one common way of reducing basis set superposition error (BSSE) is to use counterpoise corrections by employing ghost functions on ghost atoms. A useful guide to BSSE can be found [here](http://vergil.chemistry.gatech.edu/notes/cp.pdf). In PySCF, counterpoise corrections can be applied via simple modifications to the atom and basis attributes.

For example, running the water dimer calculation with ghost functions on the second water monomer can be accomplished via:
```
mol.atom="""
O  -1.551007  -0.114520   0.000000
H  -1.934259   0.762503   0.000000
H  -0.599677   0.040712   0.000000
GhostO   1.350625   0.111469   0.000000
GhostH   1.680398  -0.373741  -0.758561
GhostH   1.680398  -0.373741   0.758561"""
mol.basis='aug-cc-pVDZ'
```
In this case, the parser understands that the latter three atoms are meant to be ghost atoms since the atomic symbol is prepended with the string, 'Ghost'.

Advanced users who are interested in using custom basis sets or basis sets that not pre-defined in PySCF are directed [here](http://sunqm.github.io/pyscf/gto.html#input-basis). Useful resources for the latter include the [EMSL basis set exchange](https://bse.pnl.gov/bse/portal), the [PSI4 basis set page](http://www.psicode.org/psi4manual/master/basissets_byfamily.html), Grant Hill's [ccRepo](http://www.grant-hill.group.shef.ac.uk/ccrepo/), and the [TURBOMOLE basis set library](http://www.cosmologic-services.de/basis-sets/basissets.php). PySCF can parse NWChem- and Gaussian94-formatted basis sets [TODO].

For example, the STO-2G basis set is not pre-defined in PySCF. If one wants to use the STO-2G basis set for the hydrogen atoms and the STO-3G basis set for the oxygen atoms, then the appropriate basis setup would be:
```
mol.basis={'H': gto.basis.parse('''
H    S
      1.309756377            0.430128498      
      0.233135974            0.678913531      
'''),'O': 'STO-3G'}
```

Additionally, PySCF can read a file containing the desired basis set [TODO]:
```
mol.basis={'H': gto.basis.read('basis/STO-2G.dat'),'O': 'STO-3G'}
```

## cart attribute
#### DEFAULT: ```mol.cart=0```

The cart attribute determines whether the d and/or higher basis functions are taken to be *spherical* (i.e., d=5 functions, f=7 functions, g=9 functions, etc.) or *Cartesian* (i.e., d=6 functions, f=10 functions, g=15 functions, etc.). ```mol.cart=0``` specifices *spherical* functions and ```mol.cart=1``` specifies *Cartesian* functions. If an existing basis set in PySCF is being used, the cart attribute is automatically set based on how the basis set itself was optimized. Thus, it is not necessary to define this attribute for most calculations.

There are two instances where specifying the cart attribute is necessary:

1.  If either a custom basis set is being used or one that is not pre-defined in PySCF, it is important to determine the appropriate value of cart and set it accordingly. For example, the [UGBS basis set](https://aip.scitation.org/doi/abs/10.1063/1.475959) is commonly used to determine absolute atomic energies. This basis set was optimized for use with spherical functions. Since UGBS is not pre-defined in PySCF, it is important to set ```mol.cart=0```.

2.  If the user is interested in experimenting with the effect of using spherical vs. Cartesian functions for a given basis set, it is possible to override the default for pre-defined basis sets. For example, the Dunning basis sets (cc-pVXZ and aug-cc-pVXZ) are meant to be used with spherical functions (```mol.cart=0```), yet one can set ```mol.cart=1``` to gauge the sensitivity of absolute and relative energies to this setting.

## charge attribute
#### DEFAULT: ```mol.charge=0```

The charge attribute sets the charge of the molecule, and is 0 for neutral systems, 1 for cations, 2 for dications, -1 for anions, -2 for dianions, etc.

## ecp attribute
#### DEFAULT: ```mol.ecp={}```

When heavy elements are present in a molecule, standard Gaussian basis sets are oftentimes both insufficient (because they are unable to capture the relativistic nature of heavier elements) and impractical (because as one descends the rows of the periodic table, Gaussian basis sets tend to become very large and heavily-contracted due to the increased number of electrons). For this reason, a great deal of work has been devoted to developing effective core potentials (ECP) which replace core electrons around a nucleus by pseudopotentials.

In PySCF, the ecp attribute is very similar to the basis attribute. To demonstrate the use of the ecp attribute, we will switch to a system with heavy elements, namely, TeH<sub>2</sub>--HI. This system is from the [HEAVY28](http://www.thch.uni-bonn.de/tc.old/downloads/GMTKN/GMTKN55/HEAVY28.html) dataset of the [GMTKN55](https://www.chemie.uni-bonn.de/pctc/mulliken-center/software/GMTKN/gmtkn55) database. Specifically, it is [data point](http://www.thch.uni-bonn.de/tc.old/downloads/GMTKN/GMTKN55/HEAVY28ref.html) \#27, an intermolecular bond between tellerium hydride and hydrogen iodide:

<img style="float: left;" src="images/TeH2_HI.jpg" width="50%"><br clear="all" />

The atom and basis attributes can be specified as:
```
mol.atom="""
Te         0.03866320      2.16215120      0.00000000
H         -0.10453280      0.53339420      0.00000000
H         -1.58803480      2.30795520      0.00000000
H          1.61524120     -2.39760280      0.00000000
I          0.03866320     -2.60589780      0.00000000"""
mol.basis='def2-TZVPP'
```

It is possible to specify a single ECP for all atoms present in the system:
```
mol.ecp='def2-TZVPP'
```
However, if there is an atom in the system that does not have an ECP defined (hydrogen in this case), a warning will be shown:
```
ECP def2-TZVPP not found for  H
```
Therefore, it is advisable to use the dictionary data structure to specify the ECP:
```
mol.ecp={'Te': 'def2-TZVPP', 'I': 'def2-TZVPP'}
```

It is also possible to use ECPs that are not defined in PySCF. Useful resources include the four websites listed earlier, as well as the [Stuttgart/Cologne ECP web page](http://www.tc.uni-koeln.de/PP/index.en.html). For example, if one wants to use the older def-TZVPP ECP for Te only (the newer version, def2-TZVPP, is pre-defined in PySCF), it is possible to get the neccessary information from the [TURBOMOLE basis set library](http://www.cosmologic-services.de/basis-sets/basissets.php):

```
mol.ecp={'Te': gto.basis.parse('''
TE     0
TE-ECP     3     46
f POTENTIAL
  1
2      1.93927000          -17.86464100
s-f POTENTIAL
  3
2      2.92379400           50.08380500
2      1.15275400            1.96814000
2      1.93927000           17.86464100
p-f POTENTIAL
  3
2      2.60308600          119.82070200
2      0.98544800           -2.03904800
2      1.93927000           17.86464100
d-f POTENTIAL
  2
2      1.43501900           37.75721400
2      1.93927000           17.86464100
'''),'I': 'def2-TZVPP'}
```

In addition, PySCF can read a file containing the desired ECP [TODO]:
```
mol2.ecp={'Te': gto.basis.read('ecp/def-TZVPP.dat'),'I': 'def2-TZVPP'}
```

## spin attribute
#### DEFAULT: ```mol.spin=0```

The spin attribute simply sets the number of unpaired electrons. By default, a closed-shell system is assumed: ```mol.spin=0```. For open-shell systems, the spin must be set. For example, the boron atom has a single unpaired electron: ```mol.spin=1```. The carbon atom has two unpaired electrons: ```mol.spin=2```. And finally, nitrogen has three unpaired electrons: ```mol.spin=3```. The same concept applies to open-shell molecules. For instance, the hydroxyl radical (HO•) should have ```mol.spin=1```, since it has one unpaired electron.

## unit attribute
#### DEFAULT: ```mol.charge='Angstrom'```

The unit attribute can be either set to 'Angstrom' (default) or 'Bohr'.

## verbose attribute
#### DEFAULT: ```mol.verbose=0```

The verbose attribute controls the print level for the molecule object. Setting ```mol.verbose=0``` will print little to no information, while setting ```mol.verbose=4``` prints useful information about the basis and number of basis functions [TODO]. Users who want to see detailed information should set ```mol.verbose=10```.

The final step is to build the molecule object (if any attributes are specified or modified after the molecule is built, this command should be executed again):
```
mol.build()
```

In [3]:
mol=gto.Mole()

mol.atom="""
O  -1.551007  -0.114520   0.000000
H  -1.934259   0.762503   0.000000
H  -0.599677   0.040712   0.000000
O   1.350625   0.111469   0.000000
H   1.680398  -0.373741  -0.758561
H   1.680398  -0.373741   0.758561"""
mol.basis='aug-cc-pVDZ'
mol.cart=False
mol.charge=0
mol.ecp={}
mol.spin=0
mol.unit='Angstrom'
mol.verbose=0

mol.build()

#ADVANCED ATTRIBUTES -- SEE MANUAL
#mol.groupname=
#mol.incore_anyway=
#mol.irrep_id=
#mol.irrep_name=
#mol.max_memory=
#mol.nucmod=
#mol.output=
#mol.symm_orb=
#mol.symmetry=
#mol.symmetry_subgroup=
#mol.topgroup=

---

After the molecule object has been built, it is necessary to create a mean-field object [Hartree--Fock (HF) or Kohn--Sham Density Functional Theory (KS-DFT)].

In order to create a mean-field object, it is necessary to import the scf (self-consistent field) submodule:

In [4]:
from pyscf import scf

For most users, the relevant types of SCF calculations will be:

1. RHF (Restricted Hartree--Fock)
2. UHF (Unrestricted Hartree--Fock)
3. ROHF (Restricted Open-Shell Hartree--Fock)
4. RKS (Restricted Kohn--Sham)
5. UKS (Unrestricted Kohn--Sham)
6. ROKS (Restricted Open-Shell Kohn--Sham)

Initializing the mean-field object for these six types of calculations is demonstrated below. These classes require at least one argument, namely, a molecule object, mol.

1. ```mf=scf.RHF(mol)```
2. ```mf=scf.UHF(mol)```
3. ```mf=scf.RHF(mol)``` (where mol is defined to be open-shell, mol.spin!=0)
4. ```mf=scf.RKS(mol)```
5. ```mf=scf.UKS(mol)```
6. ```mf=scf.RKS(mol)``` (where mol is defined to be open-shell, mol.spin!=0)

# Creating a mean-field (mf) object with scf.RHF()

Since RHF, ROHF, and UHF have nearly identical attributes, the tutorial will focus on an RHF calculations. The first step is to create the method object (RHF in this case):
```
mf=scf.RHF(mol)
```
The input object is the molecule (mol) that was discussed previously.

## conv_tol attribute
#### DEFAULT: ```mf.conv_tol=1e-09```
In order for an SCF calculation to converge, PySCF requires two criteria to be met. The first is controlled by the conv_tol attribute, namely, the difference in the SCF energy between two sucessive cycles.

## conv_tol_grad attribute
#### DEFAULT: ```mf.conv_tol_grad=numpy.sqrt(mf.conv_tol)```
The second criterion for convergence is the conv_tol_grad attribute, namely, the root-mean-square [TODO] of the orbital gradient. This is a vector of length nocc\*nvirt.

## direct_scf_tol attribute
#### DEFAULT: ```mf.direct_scf_tol=1e-13```
The direct_scf_tol attribute is the threshold for discarding integrals.

## init_guess attribute
#### DEFAULT: ```mf.init_guess='minao'```
A good initial guess is vital for the efficient completion of the SCF procedure. There are four initial guess options in PySCF:
1. minao
2. atom
3. 1e
4. chkfile

The first option generates an initial guess for the density matrix based on the ANO basis, and then projects this onto the basis set specified in ```mol.basis```.

The second option generates an initial guess based on a superposition of atomic densities computed in the basis set specified in ```mol.basis```.

The third option sets the initial density matrix to zero. This is generally a very poor guess and should only be used for debugging or checking values between different packages.

The last option is somewhat advanced and reads in an existing density matrix from disk. Further information on this option can be found [here](http://sunqm.github.io/pyscf/scf.html#pyscf.scf.hf.SCF.init_guess_by_chkfile).

## max_cycle attribute
#### DEFAULT: ```mf.max_cycle=50```
The max_cycle attribute simply sets the maximum number of SCF cycles that should be performed before the calculation terminates. For systems that are notoriously difficult to converge, the value should be increased to 100 or even 1000.

## max_memory attribute
#### DEFAULT: ```mf.max_memory=4000```
The max_memory attribute determines the maximum amount of memory (in Megabytes) that PySCF is allowed to utilize during the SCF procedure. This should be set by the user based on the memory limitations of the computer/server utilized.

## verbose attribute
#### DEFAULT: ```mf.verbose=0```
The verbose attribute controls the print level for the RHF object. Setting mol.verbose=0 will print only the SCF energy, while setting mol.verbose=4 prints useful information about SCF settings as well as the SCF energy per iteration, HOMO/LUMO energies, and convergence metris. Users who want to see detailed information should set mol.verbose=10, which provides additional information such as molecular orbital energies.

The final step is to run the SCF calculation:
```
mf.kernel()
```

---

After the SCF kernel has finished running, the mean-field object is updated with several useful output attributes:
1. ```mf.mo_coeff``` - Molecular orbital (MO) coefficients (matrix where rows are atomic orbitals (AO) and columns are MOs)
2. ```mf.mo_energy``` - MO energies (vector with length equal to number of MOs)
3. ```mf.mo_occ``` - MO occupancy (vector with length equal to number of MOs)
4. ```mf.e_tot``` - Total SCF energy in units of Hartrees
5. ```mf.converged``` - Status of SCF convergence (True indicates converged and False indicates unconverged)

In [5]:
mf=scf.RHF(mol)

mf.conv_tol=1e-12
mf.conv_tol_grad=1e-8
mf.direct_scf_tol=1e-13
mf.init_guess='atom'
mf.max_cycle=100
mf.max_memory=8000
mf.verbose=0

mf.kernel()

#ADVANCED ATTRIBUTES -- SEE MANUAL
#mf.chkfile=
#mf.conv_check=
#mf.damp=
#mf.diis=
#mf.diis_file=
#mf.diis_space=
#mf.diis_space_rollback=
#mf.diis_start_cycle=
#mf.direct_scf=
#mf.level_shift=

-152.08859934750376

---

In order to run an unrestricted Hartree--Fock calculation, the steps taken above can be followed, except the mean-field object should be initiated as:
```
mf=scf.UHF(mol)
```

The most widely-used SCF method is undoubtedly density functional theory (DFT), and running a DFT calculation in PySCF is nearly as straightforward as running a Hartree--Fock calculation.

The first step is to create the method object (RKS in this case):
```
mf=scf.RKS(mol)
```

RKS and RHF (as well as UKS and UHF) share most of their attributes, so we will only cover the additional ones that pertain specifically to DFT. These attributes primarily pertain to the exchange-correlation functional and the grid used for their numerical integration.

## xc attribute
#### (NO DEFAULT)

The xc attribute sets the density functional approximation. A vast number of exchange-correlation functionals are available for use in PySCF, essentially all of those that are available in the installed version of libxc. For example, the functionals available in libxc 4.0.4 are listed [here](https://gitlab.com/libxc/libxc/wikis/Functionals-list-4.0.4).

While the parsing rules for the xc attribute are explained in the [PySCF manual](http://sunqm.github.io/pyscf/dft.html#customizing-xc-functional), we will go through a few relevant examples here:

1. Standard (pre-coded) exchange-correlation functional

  * A classic example of this scenario is the beloved B3LYP functional. To run a B3LYP calculation, simply specify ```mf.xc='B3LYP'```.
  
  * As long as commas and operators are not used, PySCF will assume that the string indicates a full exchange-correlation functional. Other examples may include ```mf.xc='PBE'```, ```mf.xc='TPSS'```, ```mf.xc='M06-L'```, etc.

2. Combination of a single exchange functional with a single correlation functional

  * This is a popular choice, particularly for using various combinations of the older exchange and correlation functionals from the 1980s and 1990s.
  
  * In PySCF, inserting a comma into the xc attribute string indicates a separation into exchange (left of the comma) and correlation (right of the comma). For instance, if once wants to run B88 exchange with PBE correlation, it is as simple as ```mf.xc='B88,PBE'```.
  
  * Since the PBE exchange-correlation functional is a combination of PBE exchange and PBE correlation, it follows that ```mf.xc='PBE'``` and ```mf.xc='PBE,PBE'``` are exactly equivalant ways to use the PBE xc functional.
  
  * Other popular combinations of a single exchange functional with a single correlation functional include ```mf.xc='B88,LYP'```, ```mf.xc='revPBE,PBE'```, ```mf.xc='OPTX,LYP'```, etc. These combinations are historically known as BLYP, revPBE, and OLYP, and can equivalently be called as ```mf.xc='BLYP'```, ```mf.xc='revPBE'```, ```mf.xc='OLYP'```.
  
  * This option is most useful when one is interested in a unconventional combination, for example, the combination of PBE exchange with TPSS correlation, ```mf.xc='PBE,TPSS'```.

3. Single exchange functional or single correlation functional

  * Using a single exchange functional, for example, PBE exchange only, is as simple as ```mf.xc='PBE,'```. Similarly, PBE correlation only can be specified with ```mf.xc=',PBE'```. In the first case (PBE exchange only), the fact that there is a comma indicates to the parser that that a pre-coded xc functional is not being utilized. Therefore, anything to the left of the comma (PBE) is taken to be the exchange functional, while anything to the right of the comma (nothing) is taken to be the correlation functional. The reverse situation applies to the second case with PBE correlation only.

4. Custom exchange-correlation functional

  * Naturally, it is possible to fully customize the xc attribute. Once again, anything to the left of the comma is assumed to be referring to exchange, and anything to the right of the comma is assumed to be reffering to correlation.
  
  * In order to start with something simple, we will demonstrate how to specify the popular [PBE0](https://aip.scitation.org/doi/abs/10.1063/1.478522) xc functional by building it from its components (the easy way to do this is simply ```mf.xc='PBE0'```). PBE0 is comprised of 25\% exact (Hartree--Fock) exchange, 75\% PBE exchange, and 100\% PBE correlation. This can be accomplished by the following line: ```mf.xc='0.25*HF+0.75*PBE,PBE'```.
  
  * A more complicated example is that of B3LYP. B3LYP is comprised of 20\% exact (Hartree--Fock) exchange, 8\% LDA exchange, 72\% B88 exchange, 19\% VWN correlation, and 81\% LYP correlation. Unfortunately, there are 6 existing parameterization of the VWN LDA correlation functional, and this has led to much confusion through the years.
  
  * PySCF uses the VWN5 parameterization, meaning the the following two assignments are equivalent
    * mf.xc='B3LYP'
    * mf.xc='0.2\*HF+0.08\*LDA+0.72\*B88,0.19\*VWN5+0.81\*LYP'
  * Q-Chem uses the VWN1RPA parameterization:
    * mf.xc='0.2\*HF+0.08\*LDA+0.72\*B88,0.19\*VWN1RPA+0.81\*LYP'
  * Gaussian uses the VWN3 parameterization:
    * mf.xc='0.2\*HF+0.08\*LDA+0.72\*B88,0.19\*VWN3+0.81\*LYP'

## grids attribute

## nlc attribute

## nlcgrids attribute

## small_rho_cutoff attribute

In [4]:
#TODO
#bare minimum is: atom, basis, w/o it should crash
#allow mol.atom to take something like @H or !H or something to indicate Ghost
#mol.cart needs to be set based on the basis set
#be able to read mol.atom from any file, and the file can either be just the XYZ, be an actual XYZ, or be q-chem-like
#one should just be able to copy and paste from an XYZ w/o the ;. that is not commonly or ever present
#D3 HAS to be implemented, and VV10 sped up, and maybe D2 D3(BJ) D4 and all that new shit etc
#what about fragment monomoners?
#just specify a single basis for everything: mol.basis={'aug-cc-pVDZ'}
#be able to take input of Gaussian94 basis
#be able to read basis from file on disk
#maybe for mol.cart instead of 0 and 1, do spherical and Cartesian
#there should be a verbose level for Mole that doesn't print bais set coeffs and stff
#lin dep should be automatically removed!!!
#shouldn't extra cycle for SCF (conv_check) only be initiated IF there is a level shift??
#some properties should be computed for free and by default -- atomic charges, S^2, multipole moments
#stability analysis / diis/gdm
#should we be using the norm of the orbital gradient for convergence of the rms
#printing HOMO/LUMO on each cycle is unnecessary or at least we should be able to turn it off
#we should print like orbital energies at end, split into occ and virt
#what exactly is this minao guess?
#mp.frozen and cc.frozen should have a default!! like if true, it should just do something obvious
#pt2 kernal should not return t2?>!?!??!
#cc kernel shouldt return t2...
#why don't q-chem and pyscf ECP calcs match absoulte energy?
#it's not clear what the default SCF conv_tol is??? 1e-9 or 1e-10?
#get for free: S^2, dipole, mulliken population
#opitmizer optioN; diis ediis adiis newton
#breakdown of energy (nuclear rep./1e/2e/DFT/disp/PCM/EFP/total)

#group all properties into def analyze and just make it default run in kernel

#give RHF/ROHF a spin square function just to print it out so ppl dont have to compute in their head
#spin per atom for UHF calc?

#PSI4 output info:
#rotational constants?
#charge/mult/electron alph beta
#dipole in different units, both nuclear/electron/full

#for DFT mf.xc=',' in addition to 'HF' maybe we should hbe 'SR-HF' and 'LR-HF'

#NWChem
#PySCF
#TURBOMOLE
#GAMESS
#Jaguar
#MOLPRO
#ORCA
#PSI4
#Q-Chem
#Gaussian
#ACES
#TeraChem
#HyperChem

In [7]:
#mf=scf.RKS(mol)


#mf.atomic_radii=
#mf.becke_scheme=
#mf.coords=
#mf.level=
#mf.mol=
#mf.non0tab=
#mf.prune=
#mf.radi_method=
#mf.radii_adjust=
#mf.stdout=
#mf.symmetry=
#mf.verbose=
#mf.weights=